# Notebook de cliente / Experimentação do modelo

In [ ]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer
from preprocess_dataset import remove_empty_entries

import torch

## Setups

In [ ]:

CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)
data = "lener_br"
dataset = load_dataset(data)

dataset = remove_empty_entries(dataset)

In [ ]:
MAX_LEN = 256

In [ ]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=MAX_LEN, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]
n_labels = 13 #TODO: por enquanto hardcodado 
model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)
input_ids = teste['input_ids']
attention_mask = teste['attention_mask']
labels = teste['targets']

## Explorando um pouco o dataset

In [ ]:
type(dataset)

In [ ]:
dataset

In [ ]:
# dataset = remove_empty_entries(dataset)

In [ ]:
dataset

In [ ]:
max_vals = [torch.max(val['targets']) for val in pytorch_dataset_train]

In [ ]:
interval = (min(max_vals).item(), max(max_vals).item())

In [ ]:
interval

In [ ]:
saida = model(input_ids, attention_mask, labels=labels)
print(saida)

## Convertendo predições para etiquetas

Nessa sessão, vamos converter logitos em probabilidades e, então, descobrir qual a label com maior probabilidade prevista pelo modelo

In [ ]:
logits = saida['logits']

In [ ]:
logits

In [ ]:
logits = logits.squeeze()
print(logits.shape)

In [ ]:
logits

In [ ]:
from torch.nn import Sigmoid
s = Sigmoid()

In [ ]:
probs = s(logits)
print(probs)

In [ ]:
probs[0]

Como um logito funciona: Um logito retornado pelo modelo tem shape \[INPUT_SIZE, BATCH_SIZE, NUM_LABELS\]. Nesse caso, batch_size=1 então fizemos um squeeze. Então o logito tem shape \[input_size, num_labels\]. Ou seja, é uma matriz onde cada linha representa uma palavra. Se passarmos o logito por um sigmoide, **a j-ésima coluna da i-ésima linha representa a probabilidade da i-ésima palavra ser da categoria j**

In [ ]:
from utils import preds_to_label_idx

In [ ]:
preds = preds_to_label_idx(logits)

In [ ]:
preds

In [ ]:
preds.shape

## Testando o DataLoader

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 8

In [ ]:
pytorch_dataset_train

In [ ]:
teste = iter(DataLoader(pytorch_dataset_train ,batch_size=batch_size))

In [ ]:
mini_batch = next(teste)

----------------------------------------

## Testando passar um minibatch ao modelo

In [ ]:
print(mini_batch)

In [ ]:
ids = mini_batch['input_ids'].squeeze().to("cuda", dtype = torch.long) # Remove a dimensão 1 ali do meio, talvez funcione?
att = mini_batch['attention_mask'].to("cuda", dtype = torch.long)
targets = mini_batch['targets'].to("cuda", dtype = torch.long)

In [ ]:
print(ids.shape)

In [ ]:
model = model.to("cuda")

In [ ]:
outs = model(ids, att, labels=targets)

In [ ]:
outs

## Convertendo logitos para labels (Em batch)

Ja vimos como converter um output de batch_size=1 para labels. Agora vamos tentar pra um batch inteiro

In [ ]:
b_logits = outs.logits

In [ ]:
b_logits.shape

In [ ]:
from utils import batch_to_labels

In [ ]:
from torch.nn import Sigmoid

In [ ]:
s = Sigmoid()

In [ ]:
probs = s(b_logits)

In [ ]:
probs[0]

In [ ]:
from utils import preds_to_label_idx, batch_to_labels

In [ ]:
preds_to_label_idx(probs[0])

In [ ]:
outputs = batch_to_labels(b_logits)

In [ ]:
outputs

## Treinando o modelo

Usando parametros padrão por enquanto só pra ver se ta tudo indo certindo

In [ ]:
from trainner import Trainner, create_data_loader
from transformers import AdamW

### Parametros


* batch sizes: 8, 16, 32, 64, 128
* learning rates: 3e-4, 1e-4, 5e-5, 3e-5

Do paper do BERT. Vou colocar num_workers = 4 por enquanto.


Uma melhoria é esconder o data_loader de clientes. Isso é detalhe de implementaçãõ, ou deveria estar na classe NERDataset

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-4, correct_bias=False)

In [ ]:
len(pytorch_dataset_train)

In [ ]:
loader = DataLoader(pytorch_dataset_train, batch_size=8)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
tr = Trainner(device, loader, model, optimizer)

In [ ]:
losses, media = tr._train_epoch()

In [ ]:
losses

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(losses)